In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# preprocessing 1

In [ ]:
!pip install transformers
!pip install emot
!pip install contractions
!pip install symspellpy
!pip -q install evaluate
!pip install googletrans==4.0.0-rc1

In [ ]:
import tensorflow as tf
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

import re
from bs4 import BeautifulSoup
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from symspellpy import SymSpell, Verbosity
from emot.emo_unicode import UNICODE_EMOJI
from emot.emo_unicode import EMOTICONS_EMO
import nltk
nltk.download('stopwords')
nltk.download('wordnet')


In [ ]:
# 런타임 확인
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/용인시 SW 해커톤/translate_train.csv')
test = pd.read_csv('/content/drive/MyDrive/용인시 SW 해커톤/translate_test.csv')

In [ ]:
train

In [ ]:
train['sentiment'].value_counts()

In [ ]:
train.isnull().sum()

In [ ]:
"""from googletrans import Translator

translator = Translator()

def translate_to_english(text):
    if text.isascii():
        return text
    else:
        try:
            translated_text = translator.translate(text, dest='en').text
            return translated_text
        except:
            return text

# test_data['text']에 있는 각 텍스트에 대해 번역을 적용합니다.
for i, text in enumerate(test['text']):
    test['text'][i] = translate_to_english(text)

for i, text in enumerate(train['text']):
    train['text'][i] = translate_to_english(text)
"""

In [ ]:
#train.to_csv('/content/drive/MyDrive/용인시 SW 해커톤/translate_train.csv', index=False)
#test.to_csv('/content/drive/MyDrive/용인시 SW 해커톤/translate_test.csv', index=False)

In [ ]:
# 1. Converitng Emojis
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text

# 2. Lower Casing
train["text"] = train["text"].str.lower()
test["text"] = test["text"].str.lower()

# 3. Removing HTML
def remove_html(text):
    soup = BeautifulSoup(text)
    text = soup.get_text()
    return text

# 4. Expand Contractions

# 5. Removing URLs
def remove_urls(text):
    pattern = re.compile(r'https?://(www\.)?(\w+)(\.\w+)(/\w*)?')
    text = re.sub(pattern, "", text)
    return text

# 6. Removing Email IDs
def remove_emails(text):
    pattern = re.compile(r"[\w\.-]+@[\w\.-]+\.\w+")
    text = re.sub(pattern, "", text)
    return text

# 7. Removing Tweeter Mentions
def remove_mentions(text):
    pattern = re.compile(r"@\w+")
    text = re.sub(pattern, "", text)
    return text


# 8. Removing Unicode Characters
def remove_unicode_chars(text):
    text = text.encode("ascii", "ignore").decode()
    return text

# 9. Abbreviation/Acronym Disambiguation
def remove_abbreviations(text):
    text = re.sub(r"mh370", "missing malaysia airlines flight", text)
    text = re.sub(r"okwx", "oklahoma city weather", text)
    text = re.sub(r"arwx", "arkansas weather", text)
    text = re.sub(r"gawx", "georgia weather", text)
    text = re.sub(r"scwx", "south carolina weather", text)
    text = re.sub(r"cawx", "california weather", text)
    text = re.sub(r"tnwx", "tennessee weather", text)
    text = re.sub(r"azwx", "arizona weather", text)
    text = re.sub(r"alwx", "alabama Weather", text)
    text = re.sub(r"wordpressdotcom", "wordpress", text)
    text = re.sub(r"usnwsgov", "united states national weather service", text)
    text = re.sub(r"suruc", "sanliurfa", text)
    return text

# 10. Removing Punctuations
punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

def remove_punctuations(text):
    text = re.sub('[%s]' % re.escape(punctuation), " ",text)
    return text

# 11. Removing Digits or Words with Digits
def remove_digits(text):
    pattern = re.compile("\w*\d+\w*")
    text = re.sub(pattern, "",text)
    return text

# 12. Removing Extra Spaces
def remove_extra_spaces(text):
    text = re.sub(' +', ' ', text).strip()
    return text

# 13. Lemmatization
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = [lemmatizer.lemmatize(word) for word in text.split()]
    text = ' '.join(words)
    return text


# 14. Spelling Correction
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = "/content/drive/MyDrive/용인시 SW 해커톤/frequency_dictionary_en_82_765.txt"
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

def correct_spelling_symspell(text):
    words = [
        sym_spell.lookup(
            word,
            Verbosity.CLOSEST,
            max_edit_distance=2,
            include_unknown=True
            )[0].term
        for word in text.split()]
    text = " ".join(words)
    return text

# 15. Correcting Compound Words

bigram_path = "/content/drive/MyDrive/용인시 SW 해커톤/frequency_bigramdictionary_en_243_342.txt"
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

def correct_compound(text):
    words = [
        sym_spell.lookup_compound(
            word,
            max_edit_distance=2
            )[0].term
        for word in text.split()]
    text = " ".join(words)
    return text


# 16. Removing Stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])


In [ ]:
# 1. convert emojis
train["text"] = train["text"].apply(convert_emojis)
test["text"] = test["text"].apply(convert_emojis)

# 2. lowercase
train["text"] = train["text"].str.lower()
test["text"] = test["text"].str.lower()

# 3. remove html
train["text"] = train["text"].apply(remove_html)
test["text"] = test["text"].apply(remove_html)

# 4. expand contractions
# train["text"] = train["text"].apply(contractions.fix)
# 5. remove urls
train["text"] = train["text"].apply(remove_urls)
test["text"] = test["text"].apply(remove_urls)

# 6. remove e-mail ids
train["text"] = train["text"].apply(remove_emails)
test["text"] = test["text"].apply(remove_emails)

# 7. remove mentions
train["text"] = train["text"].apply(remove_mentions)
test["text"] = test["text"].apply(remove_mentions)

# 8. remove unicode chars
train["text"] = train["text"].apply(remove_unicode_chars)
test["text"] = test["text"].apply(remove_unicode_chars)

# 9. remove abbreviations
train["text"] = train["text"].apply(remove_abbreviations)
test["text"] = test["text"].apply(remove_abbreviations)

# 10. remove punctuations
train["text"] = train["text"].apply(remove_punctuations)
test["text"] = test["text"].apply(remove_punctuations)

# 11. remove digits
train["text"] = train["text"].apply(remove_digits)
test["text"] = test["text"].apply(remove_digits)

# 12. remove extra spaces
train["text"] = train["text"].apply(remove_extra_spaces)
test["text"] = test["text"].apply(remove_extra_spaces)

# 13. lemmatization
train["text"] = train["text"].apply(lemmatize_text)
test["text"] = test["text"].apply(lemmatize_text)

# 14. spelling correction
train["text"] = train["text"].apply(correct_spelling_symspell)
test["text"] = test["text"].apply(correct_spelling_symspell)

# 15. correcting compound words
train["text"] = train["text"].apply(correct_compound)
test["text"] = test["text"].apply(correct_compound)

# 16 remove stopwords
#train["text"] = train["text"].apply(remove_stopwords)
#test["text"] = test["text"].apply(remove_stopwords)



In [ ]:
test.isnull().sum()

In [ ]:
import pandas as pd

# 감정 사전 파일 불러오기
sentiment_dict_path = "/content/drive/MyDrive/용인시 SW 해커톤/sentiment_dict.csv"
sentiment_dict = pd.read_csv(sentiment_dict_path)

# train 데이터프레임을 복사하여 새로운 열을 추가할 것입니다.
train_with_sentiment_words = train.copy()

# 'text' 열의 각 문장에 대해 sentiment_dict에 있는 단어를 찾아 sentiment_word 열에 추가
def find_sentiment_words(text):
    words = text.split()
    sentiment_words = [word for word in words if word in sentiment_dict['word'].values]
    return ' '.join(sentiment_words)

train_with_sentiment_words['sentiment_word'] = train_with_sentiment_words['text'].apply(find_sentiment_words)

# 결과 확인
print(train_with_sentiment_words[['text', 'sentiment_word']])


In [ ]:
train_with_sentiment_words.isnull().sum()

In [ ]:
text_len = []
for text in train.text:
    tweet_len = len(text.split())
    text_len.append(tweet_len)

train['text_len'] = text_len

train

In [ ]:
train = train[train['text_len'] > 4]

In [ ]:
train['sentiment'].value_counts()

In [ ]:
#train.to_csv('/content/drive/MyDrive/용인시 SW 해커톤/trans_pre_train.csv')
test.to_csv('/content/drive/MyDrive/용인시 SW 해커톤/trans_pre_test.csv')